#4-4 決定木

## 概要

**決定木は、条件分岐によって分割していくことで分類問題などを解く手法です。**

決定木の条件分岐は、情報利得の最大化という考え方を用いて求められます。

情報利得の最大化とは、データ分割の前後で比較して、最もデータを綺麗に分割できる条件を求めるための方法です。
決定木は、条件分岐が可視化されるため分析結果を説明しやすく、データの前処理が少なく済むため使いやすく、実務でも好んで使用される手法です。

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_04_01.jpg?raw=true" width="640px">

## アヤメデータを用いた実装

環境設定を行います

In [0]:
# Python 2, 3 をサポートします
from __future__ import division, print_function, unicode_literals

# 標準ライブラリのインポート
import numpy as np
import os

# 乱数の固定
np.random.seed(42)

# プロットの設定
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 保存先ディレクトリの設定
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"
IMAGE_DIR = "images"

os.makedirs(os.path.join(PROJECT_ROOT_DIR, IMAGE_DIR, CHAPTER_ID), exist_ok=True)

def image_path(fig_id):
    return os.path.join(PROJECT_ROOT_DIR, IMAGE_DIR , CHAPTER_ID, fig_id)

def save_fig(fig_id, tight_layout=True):
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(image_path(fig_id) + ".png", format='png', dpi=300)

決定木を理解するために、決定木を作ってどのように予測を行うのかを見てましょう。

scikit-learn に実装されている決定木分析クラス(DecisionTreeClassifier)とiris (あやめ) データセットを使って訓練します。

iris（あやめ）は、セトサ（Iris-Setosa）、バージカラー（Iris-Versicolor）、バージニカ（Iris-Virginica）の3 種類のあやめのがく片（sepal）と花弁（petal）の幅と長さが収められたデータセットです。(前段で説明)

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_06_03.jpg?raw=true" width="460px">

In [0]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # 花弁の長さと幅
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X, y)

訓練した決定木は、export_graphviz() メソッドでiris_tree.dot というグラフ定義ファイルを出力すると、可視化できます。

In [0]:
from sklearn.tree import export_graphviz

export_graphviz(
        tree_clf,
        out_file=image_path("iris_tree.dot"),
        feature_names=iris.feature_names[2:],
        class_names=iris.target_names,
        rounded=True,
        filled=True
    )

Iris データセットで構築された決定木

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_06_02.jpg?raw=true" width="320px">

木がどのように予測をするのかを見てみましょう。あやめの花を見つけ、分類したいと思ったとします。

ルートノード（root node、図の最上部で深さ0）からスタートする。このノードは、花の花弁の長さが2.45cm 未満かどうかを尋ねてきます。答えがイエスなら、ルートノードの左側の子ノードに移る（深さ1、左）、この場合、そこは葉ノード（leaf node、子ノードがないノードのこと）であるため、それ以上質問はありません。単純に予測されたクラスを見ると、決定木はその花をセトサ種（class=setosa）だと予測しています。

また別の花を見つけ、今度は花弁の長さが2.45cm よりも長いものとします。すると、今度はルートの右側の子ノードに移らなければなりません（深さ1、右）。このノードは葉ノードではなく、花弁の幅は1.75cm 未満かどうかという別の質問があります。1.75cm 未満なら、その端はたぶんバージカラーであります（深さ2、左）。そうでなければ、バージニカである可能性が高くなります（深さ2、右）。

　　　
ノードのsamples 属性は、そのノードが何個の訓練インスタンスを処理したかを示しています。

たとえば、100 個の訓練インスタンスの花弁の長さが2.45cm 以上で（深さ1、右）、そのなかの54 個の花弁の幅が1.75cm 未満（深さ2、左）だとすると、samples は図に書かれている通りになります。
ノードのvalue 属性は、各クラスの訓練インスタンスのうち何個がこのノードの条件に当てはまるかを示します。

たとえば、右下のノードの条件が当てはまるのは、セトサのなかで0 個、バージカラーのなかで1 個、バージニカのなかで45 個だということを示しています。

最後に、ノードのgini 属性はノードの不純度（impurity）を示しています。ノードの条件に当てはまるすべての訓練インスタンスが同じクラスに属するなら、そのノードは「純粋」（gini=0）であります。たとえば、深さ1、左のノードに当てはまるのはセトサ種の訓練インスタンスだけなので純粋であり、gini 係数は0 になります。

たとえば、深さ2、左ノードのgini 係数は、計算すると0.168になります。

In [0]:
from matplotlib.colors import ListedColormap

def plot_decision_boundary(clf, X, y, axes=[0, 7.5, 0, 3], iris=True, legend=False, plot_training=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if not iris:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    if plot_training:
        plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", label="Iris-Setosa")
        plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", label="Iris-Versicolor")
        plt.plot(X[:, 0][y==2], X[:, 1][y==2], "g^", label="Iris-Virginica")
        plt.axis(axes)
    if iris:
        plt.xlabel("Petal length", fontsize=14) #花弁の長さ
        plt.ylabel("Petal width", fontsize=14)  #花弁の幅
    else:
        plt.xlabel(r"$x_1$", fontsize=18)
        plt.ylabel(r"$x_2$", fontsize=18, rotation=0)
    if legend:
        plt.legend(loc="lower right", fontsize=14)

plt.figure(figsize=(8, 4))
plot_decision_boundary(tree_clf, X, y)
plt.plot([2.45, 2.45], [0, 3], "k-", linewidth=2)
plt.plot([2.45, 7.5], [1.75, 1.75], "k--", linewidth=2)
plt.plot([4.95, 4.95], [0, 1.75], "k:", linewidth=2)
plt.plot([4.85, 4.85], [1.75, 3], "k:", linewidth=2)
plt.text(1.40, 1.0, "Depth=0", fontsize=15)
plt.text(3.2, 1.80, "Depth=1", fontsize=13)
plt.text(4.05, 0.5, "(Depth=2)", fontsize=11)

save_fig("decision_tree_decision_boundaries_plot")
plt.show()

上のプロットは、決定木の決定境界を示しています。

太い縦線は、ルートノード（深さ0）の花弁長=2.45cm という決定境界を示しています。左側の領域は純粋（セトサ種だけ）のため、これ以上分割できません。しかし、右側は純粋なので、深さ1、右のノードが花弁幅1.75cm（破線で示されたもの）で分割しています。max_depth が2 に設定されているため、決定木はそこで止まっています。max_depth を3 にすることで、深さ2 のふたつのノードは、もうひとつの決定境界（点線で示されたもの）を設けていたでしょう。

**クラスの確率の推計**


決定木は、インスタンスが特定のクラスkに属する確率も推計できます。まず決定木は木構造をたどって当該インスタンスの葉ノードを見つけ、次にこのノードにあるクラスk の訓練インスタンスの割合を返します。たとえば、花弁の長さが5cm で幅が1.5cm のあやめを見つけたとします。対応する葉ノードは、深さ2、左ノードなので、決定木はセトサの確率0%（0/54）、バージカラーの確率90.7%（49/54）、バージニカの確率9.3%（5/54）を出力します。、決定木にクラスの予測を求めれば、確率がもっとも高いバージカラー（クラス1）を出力します。これをチェックしてみましょう。

In [0]:
tree_clf.predict_proba([[5, 1.5]])

In [0]:
tree_clf.predict([[5, 1.5]])

**正則化ハイパーパラメータ**


決定木は、訓練データに対してほとんど先入観（assumption）を持ちません（たとえば線形モデルなら、データが線形に分布していると最初から決めつけている）。制約を設けなければ、木構造は訓練データに合わせて自らを調整し、訓練データに密接に適合する。つまり、過学習しやすくなります。このようなモデルは、ノンパラメトリックモデル（nonparametric model）と呼ばれています。
パラメータを持たないからではなく（むしろたくさんあることが多い）、訓練に先立ってパラメータの数が決定しておらず、モデル構造がデータに密接に適合できるからであります。それに対し、線形モデルなどのパラメトリックモデル（parametric model）は、あらかじめ決められた数のパラメータがあるため、自由度が制限され、過学習のリスクが低くなっています（しかし、過小適合のリスクは高くなっている）。


訓練データへの過学習を防ぐためには、決定木の訓練中の自由に制限を加える必要があります。正則化と呼ばれるものであり、正則化ハイパーパラメータは、使うアルゴリズムによって異なるが、一般に少なくとも決定木の深さの上限は制限できます。scikit-learn では、max_depth ハイパーパラメータで設定できます（デフォルトは、無制限という意味のNone である）。
max_depth を制限すると、モデルが正則化され、過学習のリスクが低くなります。


DecisionTreeClassifier クラスは、ほかにも決定木の形に制限を加えるパラメータを持っています。min_samples_leaf（ ノードを分割するために必要なサンプル数の下限）、min_samples_leaf（葉ノードが持たなければならないサンプル数の下限）、min_weight_fraction_leaf（min_samples_leaf と同じだが、重みを持つインスタンスの
総数の割合で表現される）、max_leaf_nodes（葉ノードの数の上限）、max_features（各ノードで分割のために評価されるフィーチャー数の上限）がそうであり、min_*ハイパーパラメータを増やすかmax_*ハイパーパラメータを減らせば、モデルを正則化できます。



In [0]:
from sklearn.datasets import make_moons
Xm, ym = make_moons(n_samples=100, noise=0.25, random_state=53)

deep_tree_clf1 = DecisionTreeClassifier(random_state=42)
deep_tree_clf2 = DecisionTreeClassifier(min_samples_leaf=4, random_state=42)
deep_tree_clf1.fit(Xm, ym)
deep_tree_clf2.fit(Xm, ym)

plt.figure(figsize=(11, 4))
plt.subplot(121)
plot_decision_boundary(deep_tree_clf1, Xm, ym, axes=[-1.5, 2.5, -1, 1.5], iris=False)
plt.title("No restrictions", fontsize=16)
plt.subplot(122)
plot_decision_boundary(deep_tree_clf2, Xm, ym, axes=[-1.5, 2.5, -1, 1.5], iris=False)
plt.title("min_samples_leaf = {}".format(deep_tree_clf2.min_samples_leaf), fontsize=14)

save_fig("min_samples_leaf_plot")
plt.show()

In [0]:
angle = np.pi / 180 * 20
rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
Xr = X.dot(rotation_matrix)

tree_clf_r = DecisionTreeClassifier(random_state=42)
tree_clf_r.fit(Xr, y)

plt.figure(figsize=(8, 3))
plot_decision_boundary(tree_clf_r, Xr, y, axes=[0.5, 7.5, -1.0, 1], iris=False)

plt.show()

上結果は、moons データセット（make_moons）と呼ばれるデータで訓練したふたつの決定木を示しています。

左側の決定木はデフォルトのハイパーパラメータ（つまり制限なし）で訓練されているのに対し、右側の決定木はmin_samples_leaf=4 を指定して訓練しています。左側のモデルが過学習していることはかなりはっきりしているが、右側のモデルはそれよりも汎化性能が高いとみえます。

**回帰**

決定木は回帰のタスクも実行できます。ノイズのある2 次関数データセットを使って、max_depth=2 を指定したscikit-learn のDecisionTreeRegressor クラスを訓練し、回帰木を作ってみましょう。

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_06_04.jpg?raw=true" width="320px">

こちらの木は、今までに作ってきた分類のための木とよく似ているように見えます。最大の違いは、各ノードがクラスではなく値を予測していることであります。

たとえば、x1 = 0.6 の新インスタンスを予測
にかけたとする。ルートから木をたどっていくと、value=0.1106 を予測する葉ノードに達します。この予測は、この葉ノードに達した110 個の訓練インスタンスのターゲット値の平均に過ぎません。予測値の110 個の訓練インスタンスに対する平均二乗誤差は、0.0151 となります。
このモデルの予測は、プロット結果の左に表されています。

max_depth=3 を指定すると、右側に表すような予測が得られます。各リージョンの予測値がそのリージョン内のインスタンスのターゲット値を平均したものだということに注意しましょう。アルゴリズムは、ほとんどの訓練インスタンスが予測値にできる限り近くなるようにリージョンを分割しています。


CART アルゴリズムは、分類のときとほとんど同じように機能していますが、不純度ではなくMSE が最小になるように訓練セットを分割しているところだけが異なります。

In [0]:
# 2次トレーニングセット + ノイズ
np.random.seed(42)
m = 200
X = np.random.rand(m, 1)
y = 4 * (X - 0.5) ** 2
y = y + np.random.randn(m, 1) / 10

In [0]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg.fit(X, y)

In [0]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(random_state=42, max_depth=2)
tree_reg2 = DecisionTreeRegressor(random_state=42, max_depth=3)
tree_reg1.fit(X, y)
tree_reg2.fit(X, y)

def plot_regression_predictions(tree_reg, X, y, axes=[0, 1, -0.2, 1], ylabel="$y$"):
    x1 = np.linspace(axes[0], axes[1], 500).reshape(-1, 1)
    y_pred = tree_reg.predict(x1)
    plt.axis(axes)
    plt.xlabel("$x_1$", fontsize=18)
    if ylabel:
        plt.ylabel(ylabel, fontsize=18, rotation=0)
    plt.plot(X, y, "b.")
    plt.plot(x1, y_pred, "r.-", linewidth=2, label=r"$\hat{y}$")

plt.figure(figsize=(11, 4))
plt.subplot(121)
plot_regression_predictions(tree_reg1, X, y)
for split, style in ((0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")):
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
plt.text(0.21, 0.65, "Depth=0", fontsize=15)
plt.text(0.01, 0.2, "Depth=1", fontsize=13)
plt.text(0.65, 0.8, "Depth=1", fontsize=13)
plt.legend(loc="upper center", fontsize=18)
plt.title("max_depth=2", fontsize=14)

plt.subplot(122)
plot_regression_predictions(tree_reg2, X, y, ylabel=None)
for split, style in ((0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")):
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
for split in (0.0458, 0.1298, 0.2873, 0.9040):
    plt.plot([split, split], [-0.2, 1], "k:", linewidth=1)
plt.text(0.3, 0.5, "Depth=2", fontsize=13)
plt.title("max_depth=3", fontsize=14)

save_fig("tree_regression_plot")
plt.show()

決定木は、回帰でも分類のときと同じように過学習しがちとなります。正則化しなければ（つまり、デフォルトのハイパーパラメータを使えば）、下プロット結果の左側のグラフのような予測が得られます。これ
は訓練セットを過学習しているためです。min_samples_leaf=10 を設定するだけで、右側のグラフようなまともなモデルが得られます。

In [0]:
export_graphviz(
        tree_reg1,
        out_file=image_path("regression_tree.dot"),
        feature_names=["x1"],
        rounded=True,
        filled=True
    )

In [0]:
tree_reg1 = DecisionTreeRegressor(random_state=42)
tree_reg2 = DecisionTreeRegressor(random_state=42, min_samples_leaf=10)
tree_reg1.fit(X, y)
tree_reg2.fit(X, y)

x1 = np.linspace(0, 1, 500).reshape(-1, 1)
y_pred1 = tree_reg1.predict(x1)
y_pred2 = tree_reg2.predict(x1)

plt.figure(figsize=(11, 4))

plt.subplot(121)
plt.plot(X, y, "b.")
plt.plot(x1, y_pred1, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([0, 1, -0.2, 1.1])
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", fontsize=18, rotation=0)
plt.legend(loc="upper center", fontsize=18)
plt.title("No restrictions", fontsize=14)

plt.subplot(122)
plt.plot(X, y, "b.")
plt.plot(x1, y_pred2, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([0, 1, -0.2, 1.1])
plt.xlabel("$x_1$", fontsize=18)
plt.title("min_samples_leaf={}".format(tree_reg2.min_samples_leaf), fontsize=14)

save_fig("tree_regression_regularization_plot")
plt.show()

**不安定性**

決定木には長所がたくさんあると感じられるかと思います。決定木は理解、解釈しやすく、使いやすく、柔軟で、強力であります。しかし、決定木にも欠点はいくつかあります。

まず第1 に、決定木の決定境界は直交するものになります（すべての分割が横軸に対して垂直になっている）。そのため、訓練セットの回転によって結果が大きく変わります。

たとえば、下プロット結果に単純な線形分割可能なデータセットを示しています。左側は決定木で簡単に分割できているのに、データセットを45 度回転した右側の決定境界は不必要に入り組んでいるように見えます。どちらの決定木も訓練セットに完璧に適合しているが、右側のモデルはうまく汎化しない可能性が高くなります。この問題は、訓練データをよりよい向きに変えられることが多いPCAを使えば、ある程度軽減できます。


In [0]:
np.random.seed(6)
Xs = np.random.rand(100, 2) - 0.5
ys = (Xs[:, 0] > 0).astype(np.float32) * 2

angle = np.pi / 4
rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
Xsr = Xs.dot(rotation_matrix)

tree_clf_s = DecisionTreeClassifier(random_state=42)
tree_clf_s.fit(Xs, ys)
tree_clf_sr = DecisionTreeClassifier(random_state=42)
tree_clf_sr.fit(Xsr, ys)

plt.figure(figsize=(11, 4))
plt.subplot(121)
plot_decision_boundary(tree_clf_s, Xs, ys, axes=[-0.7, 0.7, -0.7, 0.7], iris=False)
plt.subplot(122)
plot_decision_boundary(tree_clf_sr, Xsr, ys, axes=[-0.7, 0.7, -0.7, 0.7], iris=False)

save_fig("sensitivity_to_rotation_plot")
plt.show()

より一般的に言うと、決定木の最大の問題は、訓練セットの小さな変化に敏感すぎることであります。たとえば、iris 訓練セットから花弁の幅がもっとも長いバージカラー（花弁の長さが4.8cm、幅
が1.8cm のもの）を取り除いて新しい決定木を訓練すると、下記プロットのようなモデルが得られます。これは最初の決定木と比べると大きく異なります。

実際、scikit-learn が使っている訓練アルゴリズムは確率的であるため、同じ訓練データを使っていても大きく異なるモデルが作られる
ことがあります（random_state ハイパーパラメータを設定していなければ）。
ランダムフォレストは多数の木の予測を平均するため、このような不安
定性の問題を軽減できる可能性があります。

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_06_05.jpg?raw=true" width="640px">